In [36]:
import json
import requests
import pandas as pd
import numpy as np

import sqlite3
from uszipcode import ZipcodeSearchEngine

In [37]:
conn = sqlite3.connect('us_data3.sqlite')

# us_data_df = pd.read_sql("department_contracts", conn)


us_data_df = pd.read_sql_query("SELECT * FROM department_contracts", conn)

In [38]:
us_data_df.head()


,index,Awarding_Agency,Category,POP_City,POP_State,POP_Zip,Recipient_Name,Subtier_Agency,Subtier_Code,Total_Obligation
0,0,Department of Transportation,contract,WASHINGTON,DISTRICT OF COLUMBIA,20591,EVANS INCORPORATED,Immediate Office of the Secretary of Transport...,6901,745068.00
1,1,Department of Transportation,contract,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,520169.00
2,2,Department of Transportation,contract,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,662420.00
3,3,Department of Transportation,contract,ORLANDO,FLORIDA,32822,"SIMCOM INTERNATIONAL, INC.",Federal Aviation Administration,6920,39520.00
4,4,Department of Transportation,contract,MOJAVE,CALIFORNIA,93501,NATIONAL TEST PILOT SCHOOL INC,Federal Aviation Administration,6920,29950.00


In [72]:
zip_code = pd.DataFrame(us_data_df['POP_Zip'].unique())
zip_code['Latitude'] = ''
zip_code['Longitude'] = ''
zip_code.columns = [
    'POP_Zip',
    'Latitude',
    'Longitude'
]

zip_code.head()



,POP_Zip,Latitude,Longitude
0,20591,,
1,21230,,
2,32822,,
3,93501,,
4,11371,,


In [56]:
test = {'Zip':['94102', '21230','32822','93850','11371'],
        'Latitude':['','','','',''],
        'Longitude':['','','','','']}
test_df = pd.DataFrame(test)
test_df

,Latitude,Longitude,Zip
0,,,94102
1,,,21230
2,,,32822
3,,,93850
4,,,11371


In [46]:

search = ZipcodeSearchEngine()

for index, row in test_df.iterrows():    
    zipcode = search.by_zipcode(row['Zip']) 
    row['Latitude'] = zipcode['Latitude']
    row['Longitude'] = zipcode['Longitude'] 
    
# for index, row in us_data_df.iterrows():    
#     if row['Latitude'] == None:
#         res = search.by_city(row['POP_City'])
#         row['Latitude']= res[0]['Latitude']
#         row['Longitude'] = res[0]['Longitude']

test_df

,Latitude,Longitude,Zip
0,37.7787,-122.421,94102
1,39.2635,-76.6239,21230
2,28.475,-81.2849,32822
3,None,None,93850
4,40.7766,-73.8743,11371


In [47]:
us_data_df['Latitude'] = ''
us_data_df['Longitude'] = ''
us_data_df.head()

,index,Awarding_Agency,Category,POP_City,POP_State,POP_Zip,Recipient_Name,Subtier_Agency,Subtier_Code,Total_Obligation,Latitude,Longitude
0,0,Department of Transportation,contract,WASHINGTON,DISTRICT OF COLUMBIA,20591,EVANS INCORPORATED,Immediate Office of the Secretary of Transport...,6901,745068.00,,
1,1,Department of Transportation,contract,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,520169.00,,
2,2,Department of Transportation,contract,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,662420.00,,
3,3,Department of Transportation,contract,ORLANDO,FLORIDA,32822,"SIMCOM INTERNATIONAL, INC.",Federal Aviation Administration,6920,39520.00,,
4,4,Department of Transportation,contract,MOJAVE,CALIFORNIA,93501,NATIONAL TEST PILOT SCHOOL INC,Federal Aviation Administration,6920,29950.00,,


In [ ]:
search = ZipcodeSearchEngine()

for index, row in us_data_df.iterrows():    
    zipcode = search.by_zipcode(row['POP_Zip'])
    zip_lat = zipcode['Latitude']
    zip_lon = zipcode['Longitude']
    
    if zip_lat == None:
        res = search.by_city(row['POP_City'])
        try:
            row['Latitude']= res[0]['Latitude']
        except:
            row['Latitude']= None
        try:
            row['Longitude']= res[0]['Longitude']
        except:
            row['Longitude']= None
    else:       
        row['Latitude'] = zip_lat
        row['Longitude'] = zip_lon
    
# for index, row in us_data_df.iterrows():    
#     if row['Latitude'] == None:
#         res = search.by_city(row['POP_City'])
#         row['Latitude']= res[0]['Latitude']
#         row['Longitude'] = res[0]['Longitude']

conn = sqlite3.connect('final_dataset.sqlite')

us_data_df.to_sql("department_contracts", conn, if_exists="replace")